In [20]:
import pandas as pd
import numpy as np
import glob
import os
import seaborn as sns
from matplotlib import pyplot as plt 
from sklearn import preprocessing
import statsmodels.formula.api as sm

url = 'https://github.com/AnoVando/TA1/blob/master/TA1.csv'
TA1 = pd.read_csv(url, sep='comma', engine='python')

In [22]:
TA1.head()

,<!DOCTYPE html>
0,"<html lang=""en"">"
1,<head>
2,"<meta charset=""utf-8"">"
3,"<link rel=""dns-prefetch"" href=""https://github...."
4,"<link rel=""dns-prefetch"" href=""https://avatars..."


In [21]:
# path = r'C:/Users/Joshua.Matthews/Desktop/SDXC/Joshua.Matthews/School.Documents/MSIS_521/TA1/TA1.xlsx'   

# TA1 = pd.read_excel(path)
# TA1.columns = TA1.iloc[0]
# TA1 = TA1.drop(TA1.index[[0]])
TA1.columns = ['Manufacturer', 'Screen_Size', 'Motion_Rate', 'Pixel', 'Overall_Rating', 'Video_Quality', 'Sound_Quality', 'Appearance',
              'Number_Features', 'Ease_Setup', 'Lifespan', 'Expert_Rating', 'Size_26', 'Size_32', 'Size_36', 'Size_40', 'Size_42',
              'Size_46', 'Size_49', 'Size_52', 'Size_55', 'Size_60', 'Motionrate_120', 'Motionrate_240', 'Motionrate_600',
              'Pixel_1080', "Selling_Price", 'Yearly_Units_Sold']

ValueError: Length mismatch: Expected axis has 1 elements, new values have 28 elements

In [ ]:
columns = ['Screen_Size', 'Motion_Rate', 'Pixel', 'Overall_Rating', 'Video_Quality', 'Sound_Quality', 'Appearance',
              'Number_Features', 'Ease_Setup', 'Lifespan', 'Expert_Rating', 'Size_26', 'Size_32', 'Size_36', 'Size_40', 'Size_42',
              'Size_46', 'Size_49', 'Size_52', 'Size_55', 'Size_60', 'Motionrate_120', 'Motionrate_240', 'Motionrate_600',
              'Pixel_1080', "Selling_Price", 'Yearly_Units_Sold']

TA1[columns] = TA1[columns].apply(pd.to_numeric)
TA1.head()

## Bonus Credit - Regression Using Selling Price Differential

In [113]:
TA1['Segment']=TA1['Screen_Size'].astype(str)+'_'+TA1['Motion_Rate'].astype(str)+'_'+TA1['Pixel'].astype(str)
segment_price = TA1[['Segment', 'Selling_Price']].copy()
segment_price["Selling_Price"] = segment_price["Selling_Price"].apply(pd.to_numeric)
segment_price2 = segment_price.groupby('Segment', as_index=False)['Selling_Price'].mean()
segment_price2.columns = ["Segment", "Segment_Price"]
ta1_new = pd.merge(TA1, segment_price2, on='Segment', how='inner')
ta1_new["Price_Differential"] = ta1_new["Selling_Price"] - ta1_new["Segment_Price"]
ta1_new.head()

,Manufacturer,Screen_Size,Motion_Rate,Pixel,Overall_Rating,Video_Quality,Sound_Quality,Appearance,Number_Features,Ease_Setup,...,Size_60,Motionrate_120,Motionrate_240,Motionrate_600,Pixel_1080,Selling_Price,Yearly_Units_Sold,Segment,Segment_Price,Price_Differential
0,LG,19,60,720,3.71,3.34,4.12,3.61,4.33,3.38,...,0,0,0,0,0,430,16041,19_60_720,438.75,-8.75
1,Panasonic,19,60,720,3.95,4.32,3.55,3.65,3.93,4.16,...,0,0,0,0,0,475,16777,19_60_720,438.75,36.25
2,Sony,19,60,720,4.19,3.80,4.05,3.51,2.96,4.19,...,0,0,0,0,0,575,15687,19_60_720,438.75,136.25
3,Samsung,19,60,720,4.19,4.40,4.01,4.19,4.36,3.98,...,0,0,0,0,0,540,16826,19_60_720,438.75,101.25
4,Philips,19,60,720,3.33,3.44,3.92,3.51,2.64,3.47,...,0,0,0,0,0,400,15490,19_60_720,438.75,-38.75


In [114]:
#Adding Revenue Column
ta1_new["Revenue"] = ta1_new["Selling_Price"] * ta1_new["Yearly_Units_Sold"]

In [115]:
result_ta1_new = sm.ols(formula="Revenue ~ Video_Quality+Sound_Quality+Appearance+Number_Features+Lifespan+Ease_Setup \
                        +Expert_Rating+Size_26+Size_32+Size_36+Size_40+Size_42+Size_46+Size_49+Size_52 \
                        +Size_55+Size_60+Motionrate_120+Motionrate_240+Motionrate_600+Pixel_1080",
                data=ta1_new).fit()
print(result_ta1_new.summary())

                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     1567.
Date:                Mon, 20 Jan 2020   Prob (F-statistic):               0.00
Time:                        19:14:26   Log-Likelihood:                -10465.
No. Observations:                 704   AIC:                         2.097e+04
Df Residuals:                     682   BIC:                         2.108e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept       -7.815e+06   3.16e+05    -

In [116]:
result_ta1_new = sm.ols(formula="Revenue ~ Video_Quality+Sound_Quality+Appearance+Number_Features+Lifespan+Ease_Setup \
                        +Expert_Rating+Size_26+Size_32+Size_36+Size_40+Size_42+Size_46+Size_49+Size_52 \
                        +Size_55+Size_60+Motionrate_120+Motionrate_240+Motionrate_600+Pixel_1080+Price_Differential",
                data=ta1_new).fit()
print(result_ta1_new.summary())

                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     2537.
Date:                Mon, 20 Jan 2020   Prob (F-statistic):               0.00
Time:                        19:14:31   Log-Likelihood:                -10282.
No. Observations:                 704   AIC:                         2.061e+04
Df Residuals:                     681   BIC:                         2.071e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           9.475e+05   4.73